In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd

/opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9B280146-BBD7-3F77-9873-F9740F2A5329> /opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <650A7408-F84B-37A3-B9D0-8E799E9E0543> /opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Cifar10 dataset 

In [2]:
cifar = load_dataset("cifar10",)
i = cifar["train"]["img"][0]
print(i.size)

(32, 32)


Classes extraction

In [3]:
labels = cifar["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
    
print(labels)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### Loading trained models

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


We euse train

In [5]:
model_names = ['resnet50', 'vgg16', 'densenet121']
models_to_compare = {}

for name in model_names:
    model = getattr(models, name)(pretrained=True)
    models_to_compare[name] = model

/opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weigh

We use the following function to make sure that the output is given by 10 n neurons for the 10 classes.

In [6]:
def modify_model(model, num_classes):
    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False

    # Modify the final layer based on model architecture
    if isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif isinstance(model, models.VGG):
        # For VGG models, classifier is a Sequential module
        num_ftrs = model.classifier[0].in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    elif isinstance(model, models.DenseNet):
        # For DenseNet models, classifier is a Linear layer
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
    else:
        raise NotImplementedError(f"Model architecture {type(model)} not supported")

    return model

for name in models_to_compare:
    models_to_compare[name] = modify_model(models_to_compare[name], 10)

### Dataset Class

In [7]:
transform = transforms.Compose([
    # you can add other transformations in this list
     transforms.Resize((32,32)),
    transforms.ToTensor()
])

class Dataset(Dataset):
    
    # Constructor 
    def __init__(self, X_data, Y_data, transform=transform):
        self.len = len(X_data)
        self.x = X_data
        self.y = Y_data
        self.transform = transform
             
    # Getter
    def __getitem__(self, index):
        x = self.x[index] 
        y = self.y[index]
        if self.transform:
            x = self.transform(x)     
        return x, y
    
    # Get Length
    def __len__(self):
        return self.len


In [9]:
batch_size=256

dataset_val = Dataset( X_data = cifar["train"]["img"][0:10000], Y_data = cifar["train"]["label"][0:10000])
val_loader = DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=True)
dataset_train = Dataset( X_data = cifar["train"]["img"][10000:-1], Y_data = cifar["train"]["label"][10000:-1])
train_loader = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
dataset_test = Dataset( X_data = cifar["test"]["img"], Y_data = cifar["test"]["label"])
test_loader = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=True)

Choosing the Optimizer and the Cost function

In [11]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def get_optimizer(model):
    # Only parameters of final layers are being optimized
    return optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)


### Training the model

In [15]:
import time
import copy

def train_model(model, criterion, optimizer, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    epoch_train_loss = []
    epoch_train_acc = []
    epoch_val_loss = []
    epoch_val_acc = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        ###### TRAINING #######
        print("Training")
        model.train()

        train_losses = []
        train_acc = []

        for x,y in train_loader:
            x, y = x.to(device), y.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(x)
            loss = criterion(output, y)
            acc = accuracy(output, y)
            loss.backward()
            optimizer.step()

            # Statistics
            train_losses.append(loss.item())
            train_acc.append(acc.item())
        
        epoch_train_loss.append(sum(train_losses)/len(train_losses))
        epoch_train_acc.append(sum(train_acc)/len(train_acc))

        ### VALIDATION ###
        print("Validation")
        val_losses = []
        val_acc = []
        val_loss_total = 0

        model.eval()
        with torch.no_grad():
            for x,y in val_loader:

                x = x.to(device)
                y = y.to(device)
                output = model(x)
                loss = criterion(output, y)
                acc = accuracy(output, y)
                
                val_losses.append(loss.item())
                val_acc.append(acc.item())
                
        epoch_val_loss.append(sum(val_losses)/len(val_losses))
        epoch_val_acc.append(sum(val_acc)/len(val_acc))

        print(f"Valid epoch {epoch} loss:", "{:.4f}".format(sum(val_losses)/len(val_losses)), "acc", "{:.4f}".format(sum(val_acc)/len(val_acc)),"\n")
        print()
        val_loss_total = sum(val_losses)/len(val_losses)
        if val_loss_total < best_loss:
            best_loss = val_loss_total
            best_model_wts = copy.deepcopy(model.state_dict())
            patience = 3  
        else:
            patience -= 1
            if patience == 0:
                break   
            
    model.load_state_dict(best_model_wts)
    return model


In [16]:
# Train each model
trained_models = {}
for name, model in models_to_compare.items():
    print(f'\nTraining {name} model...')
    optimizer = get_optimizer(model)
    trained_models[name] = train_model(model, criterion, optimizer, num_epochs=3)



Training resnet50 model...
Epoch 0/2
----------
Training
Validation
Valid epoch 0 loss: 1.5187 acc 0.4829 


Epoch 1/2
----------
Training
Validation
Valid epoch 1 loss: 1.4780 acc 0.4953 


Epoch 2/2
----------
Training
Validation
Valid epoch 2 loss: 1.4726 acc 0.4997 



Training vgg16 model...
Epoch 0/2
----------
Training
Validation
Valid epoch 0 loss: 1.1962 acc 0.5783 


Epoch 1/2
----------
Training
Validation
Valid epoch 1 loss: 1.1636 acc 0.5935 


Epoch 2/2
----------
Training
Validation
Valid epoch 2 loss: 1.1523 acc 0.6002 



Training densenet121 model...
Epoch 0/2
----------
Training
Validation
Valid epoch 0 loss: 1.4878 acc 0.5054 


Epoch 1/2
----------
Training
Validation
Valid epoch 1 loss: 1.4176 acc 0.5216 


Epoch 2/2
----------
Training
Validation
Valid epoch 2 loss: 1.3671 acc 0.5374 




#### Evaluating the model on the test data

In [17]:
def evaluate_model(model):
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    test_losses = []
    test_acc = []

    with torch.no_grad():
        for x,y in test_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = criterion(output, y)
            acc = accuracy(output, y)
            
            test_losses.append(loss.item())
            test_acc.append(acc.item())

    acc = sum(test_acc)/len(test_acc)
    return acc


In [18]:

# Evaluate each model
model_performances = {}
for name, model in trained_models.items():
    acc = evaluate_model(model)
    model_performances[name] = acc
    print(f'Model {name} Accuracy: {acc:.4f}')

Model resnet50 Accuracy: 0.5131
Model vgg16 Accuracy: 0.5957
Model densenet121 Accuracy: 0.5305


In [ ]:
import pandas as pd

performance_df = pd.DataFrame(list(model_performances.items()), columns=['Model', 'Accuracy'])
print(performance_df)